In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import  Keys 
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.wait import WebDriverWait 
from selenium.webdriver.common.by import  By 
from selenium.webdriver.support import  expected_conditions as EC 

import pandas as pd
import numpy as np
import time
import datetime

In [11]:
#建立数据库
columns=['时间','发布日期','作者名','作者类型','粉丝数','投稿数','视频名','分区','子分区','视频类型','自制or转载','标签','视频长度','播放量','点赞数','投币数','收藏数','转发数','评论数','弹幕数','有无分集']
channel_yinyue_MV=pd.DataFrame(columns=columns)
channel_yinyue_MV

,时间,发布日期,作者名,作者类型,粉丝数,投稿数,视频名,分区,子分区,视频类型,...,标签,视频长度,播放量,点赞数,投币数,收藏数,转发数,评论数,弹幕数,有无分集


In [18]:
driver=webdriver.Chrome()
driver.get('https://passport.bilibili.com/login') 

In [19]:
#登录
driver.find_element_by_xpath('//*[@id="login-username"]').send_keys('15717506191')
driver.find_element_by_xpath('//*[@id="login-passwd"]').send_keys('xy000000')
driver.find_element_by_xpath('//*[@id="geetest-wrap"]/div/div[5]/a[1]').click()
#需要手动过验证码

In [5]:
driver.find_element_by_xpath('//*[@id="primaryChannelMenu"]/span[3]/div/a/span').click()  #选择“音乐”频道

In [6]:
#获得分区名
channel=driver.find_element_by_xpath('//*[@id="primaryChannelMenu"]/span[3]/div/a/span').text
channel=channel.strip('0123456789+')
channel

'音乐'

In [7]:
driver.find_element_by_xpath('//*[@id="subnav"]/ul/li[7]/a').click()  #选择“MV”子频道
sub_channel=driver.find_element_by_xpath('//*[@id="subnav"]/ul/li[7]/a').text
sub_channel

'MV'

In [8]:
driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')        
driver.find_element_by_xpath('//*[@id="videolist_box"]/div[1]/div[1]/ul/a[2]/li').click() #让视频按热度排序

In [25]:
print(datetime.datetime.now())

for month in range(4,11): 
    driver.find_element_by_xpath('//*[@id="videolist_box"]/div[1]/div[3]/div[2]/div/span').click()
    period=driver.find_element_by_xpath('//*[@id="videolist_box"]/div[1]/div[3]/div[2]/div[2]/div[2]/div/div[2]/a[%d]'%month).text
    driver.find_element_by_xpath('//*[@id="videolist_box"]/div[1]/div[3]/div[2]/div[2]/div[2]/div/div[2]/a[%d]'%month).click()
    WebDriverWait(driver,60).until(EC.presence_of_element_located((By.XPATH,'//*[@id="videolist_box"]/div[2]/ul/li[1]/div/div[2]/div[3]/a'))) 
    for page in range(5):    #爬取每个频道1月到10月每个月播放量前100的视频

        video_loc=1
        while video_loc<21:

            ### 判断作者类型###
            
            
            driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/div[3]/a'%video_loc).click()
            driver.switch_to_window(driver.window_handles[-1])
            
            
            author_web_show='No'
            try:
                i=0
                while author_web_show=='No':
                    driver.refresh()
                    i+=1
                    try:
                        WebDriverWait(driver,5).until(EC.presence_of_element_located((By.XPATH,'//*[@id="n-fs"]')))  #看作者名是否出现
                        WebDriverWait(driver,5).until(EC.presence_of_element_located((By.XPATH,'//*[@id="navigator"]/div/div[1]/div[1]/a[3]/span[3]')))  #看投稿数是否出现
                        author_web_show='Yes'
                    except:
                        author_web_show='No' 
                        if i>3:  
                            raise Exception  #刷新几次后仍爬不了说明用户已注销

                try:
                    driver.find_element_by_xpath('//*[@class="user-auth-subscript avatar-m organization-auth"]')
                    author_type='机构认证'

                except:
                    try:
                        driver.find_element_by_xpath('//*[@class="user-auth-subscript avatar-m personal-auth"]')
                        author_type='个人认证'
                    except:
                        author_type='无'

                fans_num=driver.find_element_by_xpath('//*[@id="n-fs"]').text  #获取作者粉丝数

                ### 获取作者投稿数 ###
                driver.find_element_by_xpath('//*[@class="n-num"]').click()
                WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="page-video"]/div/div[1]/div/div/ul/li[1]/span')))  
                video_num=driver.find_element_by_xpath('//*[@id="page-video"]/div/div[1]/div/div/ul/li[1]/span').text

                driver.close()
                driver.switch_to_window(driver.window_handles[-1])


                view_num=driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/div[2]/span[1]/span'%video_loc).text #获取播放量
                real_dan_mu_num=driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/div[2]/span[2]/span'%video_loc).text #获取弹幕数
                author=driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/div[3]/a'%video_loc).text #获取作者名
                driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/a'%video_loc).click() #进入视频界面  
                driver.switch_to_window(driver.window_handles[-1]) 

                try:
                    WebDriverWait(driver,5).until(EC.presence_of_element_located((By.XPATH,'//*[@id="viewbox_report"]/h1/span')))  #若出现视频被删除的情况则退出

                    video_web_show='No'
                    while video_web_show=='No':
                        driver.refresh()

                        try:
                            WebDriverWait(driver,60).until(EC.presence_of_element_located((By.XPATH,'//*[@class="bilibili-player-video-time-total"]'))) #以视频长度出现时间判定网页是否加载完
                            video_web_show='Yes'

                            try:      
                                driver.find_element_by_xpath('//*[@id="multi_page"]')   #测试是否有分集视频
                                multi='是'

                            except:
                                multi='否'

                            driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')        
                            driver.execute_script('window.scrollTo(document.body.scrollHeight,0)') #再调回最上面是因为，视频长度的元素必须在屏幕内才可以被提取，否则为空

                            bv=driver.current_url[33:44] # 获取BV号（BV号在网址中）
                            print('开始BV号为%s视频属性的抓取'%bv)
                            video_name=driver.find_element_by_xpath('//*[@id="viewbox_report"]/h1/span').text #获取视频名
                            
                            ### 获取发布日期 ###
                            release_date=driver.find_element_by_xpath('//*[@id="viewbox_report"]/div/span[3]').text
                            
                            ### 获取视频类型 ###
                            try:
                                video_type=driver.find_element_by_xpath('//*[@class="activity"]').text
                                try:
                                    driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[2]/div[1]/div[1]/span[1]')
                                    video_type=video_type+'和合作视频'
                                except:
                                    pass
                            except:
                                try:
                                    driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[2]/div[1]/div[1]/span[1]')
                                    video_type='合作视频'
                                except:
                                    video_type='普通视频'


                            ### 获取点赞数 ###
                            if driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[1]').text=='点赞':
                                like_num=0
                            else:
                                like_num=driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[1]').text 

                            ### 获取投币数 ###
                            if driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[2]').text=='投币':
                                coin_num=0
                            else:
                                coin_num=driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[2]').text

                            ### 获取收藏数 ###             
                            if driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[3]').text=='收藏':
                                collect_num=0
                            else:
                                collect_num=driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[3]').text

                            ### 获取转发数 ###       
                            if driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[4]').text=='分享':
                                forward_num=0
                            else:
                                forward_num=driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[4]').text

                            ### 获取视频长度 ###

                            video_length=driver.find_element_by_xpath('//*[@class="bilibili-player-video-time-total"]').text  

                            ### 判断自制还是转载 ###
                            driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[2]').click()
                            try:
                                driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[3]/div/div/div[2]/div[2]').text
                                self_made='自制'
                            except:
                                self_made='转载'
                            driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[3]/div/div/i').click()

                            ### 获取标签 ###
                            k=1
                            labels=np.array
                            while True:
                                try:
                                    labels=np.append(labels,driver.find_element_by_xpath('//*[@id="v_tag"]/ul/li[%d]'%k).text)
                                    k+=1
                                except: 
                                    break
                            labels=labels[1:]

                            ### 获取评论数 ###
                            driver.execute_script('window.scrollTo(0, 700)')        
                            WebDriverWait(driver,20).until(EC.presence_of_element_located((By.XPATH,'//*[@id="comment"]/div/div[1]/span[2]'))) #保证"评论"二字能加载出来                                                   
                            WebDriverWait(driver,20).until(EC.presence_of_element_located((By.XPATH,'//*[@id="comment"]/div/div[2]/div/div[4]/div')))
                            if driver.find_element_by_xpath('//*[@id="comment"]/div/div[2]/div/div[4]/div').text=='没有更多信息':    #应对评论区被关闭的情况
                                comment_num='NaN'
                            else:
                                while driver.find_element_by_xpath('//*[@id="comment"]/div/div[1]/span[1]').text=='':
                                    try:
                                        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="comment"]/div/div[2]/div/div[4]/div[1]')))
                                        while driver.find_element_by_xpath('//*[@id="comment"]/div/div[1]/span[1]').text=='':
                                            time.sleep(0.1) #很奇怪，评论数可以定位的一瞬间并不能爬取，会返回空
                                        comment_num=driver.find_element_by_xpath('//*[@id="comment"]/div/div[1]/span[1]').text     
                                    except:
                                        driver.refresh()    #有的时候“评论”二字出现，评论数不出现，页面好像卡住了
                                        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="viewbox_report"]/h1/span')))
                                        driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
                                        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="comment"]/div/div[1]/span[1]')))

                                comment_num=driver.find_element_by_xpath('//*[@id="comment"]/div/div[1]/span[1]').text

                            channel_yinyue_MV.loc[bv]=[period,release_date,author,author_type,fans_num,video_num,video_name,channel,sub_channel,video_type,self_made,labels,video_length,view_num,like_num,coin_num,collect_num,forward_num,comment_num,real_dan_mu_num,multi]

                            print('BV号为%s的视频属性抓取结束'%bv)

                            driver.close()
                            driver.switch_to_window(driver.window_handles[-1])

                        except:
                            video_web_show='No' 
                except:
                    driver.close()
                    driver.switch_to_window(driver.window_handles[-1])
            except:
                driver.close()
                driver.switch_to_window(driver.window_handles[-1])
            
            video_loc+=1


        driver.find_element_by_xpath('//*[@class="nav-btn iconfont icon-arrowdown3"]').click() #点击下一页
        WebDriverWait(driver,100).until(EC.presence_of_element_located((By.XPATH,'//*[@id="videolist_box"]/div[2]/ul/li[1]')))  #点击下一页后等页面刷新
    driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/div[2]/ul/li[2]').click()  #回到第1页
    WebDriverWait(driver,100).until(EC.presence_of_element_located((By.XPATH,'//*[@id="videolist_box"]/div[2]/div[2]/ul/li[1]')))
    driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/div[2]/ul/li[1]').click()  #发现很多月份漏爬了约20个，而且恰好是第一页，所以再点一次第一页
        
        
print(datetime.datetime.now()) 

2020-12-20 12:24:15.422885


<ipython-input-25-08981283af56>:17: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])
<ipython-input-25-08981283af56>:54: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])
<ipython-input-25-08981283af56>:61: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])


开始BV号为1gK411L7Je视频属性的抓取


E:\anaconda\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
<ipython-input-25-08981283af56>:181: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])


BV号为1gK411L7Je的视频属性抓取结束
开始BV号为1Ei4y1t7wf视频属性的抓取
BV号为1Ei4y1t7wf的视频属性抓取结束
开始BV号为1ct4y127iH视频属性的抓取
BV号为1ct4y127iH的视频属性抓取结束
开始BV号为1pi4y187Ds视频属性的抓取
BV号为1pi4y187Ds的视频属性抓取结束
开始BV号为1wi4y187g4视频属性的抓取
BV号为1wi4y187g4的视频属性抓取结束
开始BV号为1Uk4y1d7na视频属性的抓取
BV号为1Uk4y1d7na的视频属性抓取结束
开始BV号为1YK41157Rk视频属性的抓取
BV号为1YK41157Rk的视频属性抓取结束
开始BV号为1Nk4y1R7AP视频属性的抓取
开始BV号为1Nk4y1R7AP视频属性的抓取
BV号为1Nk4y1R7AP的视频属性抓取结束
开始BV号为1pz411b7iB视频属性的抓取
BV号为1pz411b7iB的视频属性抓取结束
开始BV号为1iK4y1C7vy视频属性的抓取
BV号为1iK4y1C7vy的视频属性抓取结束
开始BV号为1i54y1d73N视频属性的抓取
BV号为1i54y1d73N的视频属性抓取结束
开始BV号为1Ca4y1x7te视频属性的抓取
开始BV号为1Ca4y1x7te视频属性的抓取
BV号为1Ca4y1x7te的视频属性抓取结束
开始BV号为1wE411L7FL视频属性的抓取
BV号为1wE411L7FL的视频属性抓取结束
开始BV号为16p4y1176A视频属性的抓取
BV号为16p4y1176A的视频属性抓取结束
开始BV号为1ek4y197DP视频属性的抓取
BV号为1ek4y197DP的视频属性抓取结束
开始BV号为14Q4y1T7mT视频属性的抓取
开始BV号为14Q4y1T7mT视频属性的抓取
BV号为14Q4y1T7mT的视频属性抓取结束
开始BV号为1eK4y1r7UJ视频属性的抓取
BV号为1eK4y1r7UJ的视频属性抓取结束
开始BV号为1ii4y1t7yE视频属性的抓取
BV号为1ii4y1t7yE的视频属性抓取结束
开始BV号为19c411h72o视频属性的抓取
BV号为19c411h72o的视频属性抓取结束
开始BV号为1GK411L7ts视频属性的抓取
BV号为1GK411L7ts的视

...

开始BV号为1Ha4y1v7z3视频属性的抓取
BV号为1Ha4y1v7z3的视频属性抓取结束
开始BV号为1f54y1B7ky视频属性的抓取
BV号为1f54y1B7ky的视频属性抓取结束
开始BV号为1cZ4y1H7GB视频属性的抓取
BV号为1cZ4y1H7GB的视频属性抓取结束
开始BV号为1SD4y1D7x4视频属性的抓取
BV号为1SD4y1D7x4的视频属性抓取结束
开始BV号为12A411i7bU视频属性的抓取
开始BV号为12A411i7bU视频属性的抓取
BV号为12A411i7bU的视频属性抓取结束
开始BV号为1P5411W7YQ视频属性的抓取
BV号为1P5411W7YQ的视频属性抓取结束
开始BV号为1xV41167X5视频属性的抓取
BV号为1xV41167X5的视频属性抓取结束
开始BV号为1TT4y1J7fQ视频属性的抓取
BV号为1TT4y1J7fQ的视频属性抓取结束
开始BV号为17K4y1473x视频属性的抓取
BV号为17K4y1473x的视频属性抓取结束
开始BV号为1wz4y1R7eX视频属性的抓取
BV号为1wz4y1R7eX的视频属性抓取结束
开始BV号为1bk4y1z7Lp视频属性的抓取
BV号为1bk4y1z7Lp的视频属性抓取结束
开始BV号为1fK4y1x7oK视频属性的抓取
BV号为1fK4y1x7oK的视频属性抓取结束
开始BV号为1ei4y1G75g视频属性的抓取
BV号为1ei4y1G75g的视频属性抓取结束
开始BV号为17T4y177MK视频属性的抓取
BV号为17T4y177MK的视频属性抓取结束
开始BV号为1Ep4y1D7eG视频属性的抓取
BV号为1Ep4y1D7eG的视频属性抓取结束
开始BV号为1Jz411e7pK视频属性的抓取
BV号为1Jz411e7pK的视频属性抓取结束
开始BV号为1VD4y1Q7XV视频属性的抓取
BV号为1VD4y1Q7XV的视频属性抓取结束
开始BV号为1vC4y187gY视频属性的抓取
开始BV号为1vC4y187gY视频属性的抓取
BV号为1vC4y187gY的视频属性抓取结束
开始BV号为19T4y1J7Az视频属性的抓取
BV号为19T4y1J7Az的视频属性抓取结束
开始BV号为1na4y1v7xd视频属性的抓取
开始BV号为1na4y1v7xd

<ipython-input-25-08981283af56>:190: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])


开始BV号为1Zf4y127Co视频属性的抓取
BV号为1Zf4y127Co的视频属性抓取结束
开始BV号为1vT4y177zV视频属性的抓取
BV号为1vT4y177zV的视频属性抓取结束
开始BV号为1QD4y1D7Am视频属性的抓取
BV号为1QD4y1D7Am的视频属性抓取结束
开始BV号为1Jk4y1z7Ct视频属性的抓取
BV号为1Jk4y1z7Ct的视频属性抓取结束
开始BV号为1ET4y177ty视频属性的抓取
BV号为1ET4y177ty的视频属性抓取结束
开始BV号为19t4y1X7LJ视频属性的抓取
BV号为19t4y1X7LJ的视频属性抓取结束
开始BV号为1tV41167xC视频属性的抓取
BV号为1tV41167xC的视频属性抓取结束
开始BV号为15a4y1Y759视频属性的抓取
BV号为15a4y1Y759的视频属性抓取结束
开始BV号为1DD4y1D7Ky视频属性的抓取
BV号为1DD4y1D7Ky的视频属性抓取结束
开始BV号为1AV41167Sk视频属性的抓取
BV号为1AV41167Sk的视频属性抓取结束
开始BV号为1Qz4y197pR视频属性的抓取
BV号为1Qz4y197pR的视频属性抓取结束
开始BV号为1ED4y1S7Aw视频属性的抓取
BV号为1ED4y1S7Aw的视频属性抓取结束
开始BV号为1BC4y1874a视频属性的抓取
BV号为1BC4y1874a的视频属性抓取结束
开始BV号为1qT4y1E7Qm视频属性的抓取
BV号为1qT4y1E7Qm的视频属性抓取结束
开始BV号为1hV411r7xG视频属性的抓取
BV号为1hV411r7xG的视频属性抓取结束
开始BV号为1KK4y1b7aY视频属性的抓取
BV号为1KK4y1b7aY的视频属性抓取结束
开始BV号为1x54y1Q7w9视频属性的抓取
BV号为1x54y1Q7w9的视频属性抓取结束
开始BV号为1sv411B7KR视频属性的抓取
BV号为1sv411B7KR的视频属性抓取结束
开始BV号为1wt4y197YH视频属性的抓取
BV号为1wt4y197YH的视频属性抓取结束
开始BV号为1f54y1B77E视频属性的抓取
BV号为1f54y1B77E的视频属性抓取结束
开始BV号为1wT4y1J7K4视频属性的抓取
BV号为1wT4y1J7K4的视

开始BV号为1Lt4y1Q7hr视频属性的抓取
开始BV号为1Lt4y1Q7hr视频属性的抓取
BV号为1Lt4y1Q7hr的视频属性抓取结束
开始BV号为1qp4y1v7Rn视频属性的抓取
BV号为1qp4y1v7Rn的视频属性抓取结束
开始BV号为1ZT4y1579H视频属性的抓取
BV号为1ZT4y1579H的视频属性抓取结束
开始BV号为19T4y1577c视频属性的抓取
开始BV号为19T4y1577c视频属性的抓取
BV号为19T4y1577c的视频属性抓取结束
开始BV号为1Lh411o7GP视频属性的抓取
BV号为1Lh411o7GP的视频属性抓取结束
开始BV号为1kT4y157dA视频属性的抓取
BV号为1kT4y157dA的视频属性抓取结束
开始BV号为17V411U7hE视频属性的抓取
BV号为17V411U7hE的视频属性抓取结束
开始BV号为1E5411h7xt视频属性的抓取
BV号为1E5411h7xt的视频属性抓取结束
开始BV号为1Ea4y1E7bH视频属性的抓取
BV号为1Ea4y1E7bH的视频属性抓取结束
开始BV号为1Ft4y1Q7vo视频属性的抓取
BV号为1Ft4y1Q7vo的视频属性抓取结束
开始BV号为1fK411T75H视频属性的抓取
开始BV号为1fK411T75H视频属性的抓取
BV号为1fK411T75H的视频属性抓取结束
开始BV号为1S54y1v7eg视频属性的抓取
BV号为1S54y1v7eg的视频属性抓取结束
开始BV号为1mT4y1j7Qr视频属性的抓取
BV号为1mT4y1j7Qr的视频属性抓取结束
开始BV号为1Lh411o7uV视频属性的抓取
BV号为1Lh411o7uV的视频属性抓取结束
开始BV号为1L5411b7bk视频属性的抓取
BV号为1L5411b7bk的视频属性抓取结束
开始BV号为1Mv411i7wg视频属性的抓取
BV号为1Mv411i7wg的视频属性抓取结束
开始BV号为165411a7Zu视频属性的抓取
BV号为165411a7Zu的视频属性抓取结束
开始BV号为1Xi4y1g7dX视频属性的抓取
BV号为1Xi4y1g7dX的视频属性抓取结束
开始BV号为1Wt4y1S78N视频属性的抓取
BV号为1Wt4y1S78N的视频属性抓取结束
开始BV号为195411a7Hw

BV号为17i4y177as的视频属性抓取结束
开始BV号为1D54y1k7RS视频属性的抓取
BV号为1D54y1k7RS的视频属性抓取结束
开始BV号为1DA411L7sX视频属性的抓取
BV号为1DA411L7sX的视频属性抓取结束
开始BV号为1iv411k7F1视频属性的抓取
BV号为1iv411k7F1的视频属性抓取结束
开始BV号为1Z54y167Q8视频属性的抓取
BV号为1Z54y167Q8的视频属性抓取结束
开始BV号为18t4y1v7qi视频属性的抓取
BV号为18t4y1v7qi的视频属性抓取结束
开始BV号为1Wp4y1Y7xC视频属性的抓取
BV号为1Wp4y1Y7xC的视频属性抓取结束
开始BV号为1Sf4y1B7D6视频属性的抓取
BV号为1Sf4y1B7D6的视频属性抓取结束
开始BV号为1za4y1L7Am视频属性的抓取
BV号为1za4y1L7Am的视频属性抓取结束
开始BV号为1BK4y177s7视频属性的抓取
BV号为1BK4y177s7的视频属性抓取结束
开始BV号为1Cv41167sH视频属性的抓取
BV号为1Cv41167sH的视频属性抓取结束
开始BV号为1cr4y1w7y7视频属性的抓取
BV号为1cr4y1w7y7的视频属性抓取结束
开始BV号为1Yv411k7K1视频属性的抓取
BV号为1Yv411k7K1的视频属性抓取结束
开始BV号为1pk4y1C7Uj视频属性的抓取
BV号为1pk4y1C7Uj的视频属性抓取结束
开始BV号为1xD4y1d7V4视频属性的抓取
BV号为1xD4y1d7V4的视频属性抓取结束
开始BV号为1D54y117JG视频属性的抓取
BV号为1D54y117JG的视频属性抓取结束
开始BV号为13A411j7WG视频属性的抓取
BV号为13A411j7WG的视频属性抓取结束
开始BV号为1yz4y1Z7an视频属性的抓取
BV号为1yz4y1Z7an的视频属性抓取结束
开始BV号为1TK4y1E7jx视频属性的抓取
BV号为1TK4y1E7jx的视频属性抓取结束
开始BV号为19y4y1b7bB视频属性的抓取
BV号为19y4y1b7bB的视频属性抓取结束
开始BV号为1hz4y1o7Py视频属性的抓取
BV号为1hz4y1o7Py的视频属性抓取结束
开始BV号为1bt4y1i7bH

In [24]:
driver.switch_to_window(driver.window_handles[-1])

<ipython-input-24-dd0a1399d54d>:1: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])


In [27]:
channel_yinyue_MV.groupby('时间').agg('count')

,发布日期,作者名,作者类型,粉丝数,投稿数,视频名,分区,子分区,视频类型,自制or转载,标签,视频长度,播放量,点赞数,投币数,收藏数,转发数,评论数,弹幕数,有无分集
时间,,,,,,,,,,,,,,,,,,,,
10月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
1月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
2月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
3月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
4月,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99
5月,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99
6月,97,97,97,97,97,97,97,97,97,97,97,97,97,97,97,97,97,97,97,97
7月,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99
8月,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99


In [28]:
channel_yinyue_MV.to_json('channel_yinyue_MV.json')